<a href="https://colab.research.google.com/github/ymoslem/Adaptive-MT-LLM/blob/main/MT/Systran.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Systran Translate API

* Register to get your API key at: https://www.systran.net/
* Documentation: https://docs.systran.net/translateAPI/translation/

In [ ]:
import requests

API_KEY = "your_api_key_here"
API_URL = "https://api-translate.systran.net"
API_VERSION_ENDPOINT = "/translation/apiVersion/?key="

response = requests.get(API_URL + API_VERSION_ENDPOINT + API_KEY)
response.text

'{"version":"2.9.0"}'

In [ ]:
input1 = "Hello"
input2 = "Thanks!"

target = "fr"

API_translation_endpoint = "/translation/text/translate/"

response = requests.post(API_URL + API_translation_endpoint, data={"key": API_KEY,
                                                                   "input": [input1, input2],
                                                                   "target": target,
                                                                   })
response.text

'{"outputs":[{"output":"Bonjour"},{"output":"Merci !"}]}'

In [ ]:
import json
response_json = json.loads(response.text)

translations = [output["output"] for output in response_json["outputs"]]
translations

['Bonjour', 'Merci !']

# Full file translation

In [2]:
# Download sample files

# Source
!wget -q --show-progress https://raw.githubusercontent.com/ymoslem/Adaptive-MT-LLM/main/data/tico-19/tico-19-enfr-dedup.en
# Target
!wget -q --show-progress https://raw.githubusercontent.com/ymoslem/Adaptive-MT-LLM/main/data/tico-19/tico-19-enfr-dedup.fr

tico-19-enfr-dedup. 100%[===================>] 446.61K  --.-KB/s    in 0.005s  
tico-19-enfr-dedup. 100%[===================>] 554.30K  --.-KB/s    in 0.007s  


In [3]:
# Open files

src_file = "tico-19-enfr-dedup.en"
tgt_file = "tico-19-enfr-dedup.fr"

with open(src_file) as src, open(tgt_file) as tgt:
  source_sentences = [sent.strip() for sent in src.readlines()]
  target_sentences = [sent.strip() for sent in tgt.readlines()]

  print(source_sentences[0])
  print(target_sentences[0])

Chinese New Year (25 January) celebrations were cancelled in many places.
Les célébrations du Nouvel An chinois (le 25 janvier) ont été annulées en de nombreux endroits.


In [ ]:
# Devide a long list of source sentences into smaller chucks
# to fit into an API request

def divide_chunks(l, n):
  # looping till length l
  for i in range(0, len(l), n):
    yield l[i:i + n]

In [ ]:
# ✳️ Change the source and target language codes

source_lang = "en"
target_lang = "fr"

translations = []
for chunk_source_sentences in divide_chunks(source_sentences, 100):
  response = requests.post(API_URL + API_translation_endpoint, data={"key": API_KEY,
                                                                   "input": chunk_source_sentences,
                                                                   "source": source_lang,
                                                                   "target": target_lang,
                                                                   })
  
  if response.status_code == 200:
    response_json = json.loads(response.text)
    chunk_translations = [output["output"] for output in response_json["outputs"]]
    translations += chunk_translations
  else:
    print(response.status_code)
    break

In [ ]:
# Print the length of generated translations
print(len(translations))

3070


In [ ]:
# Print the first 10 translations
print(*translations[:10], sep="\n")

Les célébrations du Nouvel An chinois (25 janvier) ont été annulées dans de nombreux endroits.
avez-vous d'autres maladies ou problèmes médicaux chroniques comme le diabète?
Seuls quelques patients immunodéprimés ont présenté une infection sévère des voies respiratoires inférieures.
Il est fréquent que les personnes qui sont exposées à un grand nombre de virus ou dont les fonctions immunitaires sont affaiblies aient un risque plus élevé d’être infectées que les autres.
L'estimation la plus optimiste est que l'épidémie prendra fin en mars et que la phase de ralentissement durera de 3 à 4 mois.
Les cinq autres bêta-CoV comprennent HCoV-OC43, HCoV-HKU1, le coronavirus du syndrome respiratoire aigu sévère (SARS-CoV), le coronavirus du syndrome respiratoire du Moyen-Orient (MERS-CoV) et le SARS-CoV-2.
Le 26 mars, 1,7 milliard de personnes dans le monde étaient sous une forme ou une autre de confinement, qui a augmenté à 2,6 milliards de personnes deux jours plus tard, soit environ un tiers 

In [ ]:
# Save the translation file

# 🟢 Change file path and name
translations_file = "tico-19-enfr-systran.fr"

# Save translations
with open(translations_file, "w+") as output:
  for translation in translations:
    output.write(translation.strip() + "\n")